In [3]:
import pytz
import os
from datetime import datetime, timedelta
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import urllib2
import cPickle
from sunpy.time import TimeRange
from sunpy.instr import goes
from sunpy.net import hek
client = hek.HEKClient()

In [8]:
date_values = pd.date_range(pd.Timestamp('2015-03-01').tz_localize('utc'), 
                          datetime.now(pytz.utc) - MonthBegin(n=1), freq='MS')

#Get from Flares from GOES and AR from SDO

In [5]:
# main_folder_name = 'FL-GOES'
directory = 'F:\Research/Data/AR-FL/'#.format(main_folder_name)
if not os.path.exists(directory):
    os.makedirs(directory)

In [6]:
def get_goes_data(time_range):
    success_data = False
    while not success_data:
        try:
            goes_data = goes.get_goes_event_list(time_range)
            success_data = True
        except urllib2.HTTPError as err:
            pass
    return goes_data

def query_hek(tstart, tend, event_type):
    import urllib2
    success_data = False
    while not success_data:
        try:
            result_data = client.query(hek.attrs.Time(tstart,tend),hek.attrs.EventType(event_type))
            success_data = True
        except urllib2.HTTPError as err:
            pass
    return result_data

In [7]:
for d in xrange(len(date_values) - 1):
    # Get FL data from GOES
    time_range = TimeRange(date_values[d], date_values[d+1])
    goes_data = pd.DataFrame.from_dict(get_goes_data(time_range))
    goes_data['flux'] = goes_data.goes_class.apply(lambda x: goes.flareclass_to_flux(x))
    goes_data.to_csv('{0}GOES-FL-{1}.csv'.format(directory, str(date_values[d].to_period('M'))))
    print "Done FL, {0}".format(str(date_values[d].to_period('M')))

Done FL, 2015-04
Done FL, 2015-05
Done FL, 2015-06
Done FL, 2015-07
Done FL, 2015-08
Done FL, 2015-09
Done FL, 2015-10
Done FL, 2015-11
Done FL, 2015-12
Done FL, 2016-01
Done FL, 2016-02
Done FL, 2016-03
Done FL, 2016-04
Done FL, 2016-05
Done FL, 2016-06
Done FL, 2016-07
Done FL, 2016-08


In [ ]:
for d in xrange(len(date_values) - 1):
    # Get AR data
    AR_data = query_hek(date_values[d], date_values[d+1], 'AR')
    AR_data = pd.DataFrame.from_dict(AR_data)
    AR_data_selected = AR_data[[
            'ar_noaanum',
            'ar_mcintoshcls', 
            'search_frm_name',
            'search_instrument',
            'search_observatory',
            'sharp_noaa_ars',
            'obs_instrument',
            'obs_lastprocessingdate',
            'obs_levelnum',
            'obs_meanwavel',
            'obs_observatory',
            'obs_title',
            'obs_wavelunit',
            'event_coord1',
            'event_coord2',
            'boundbox_c1ll',
            'boundbox_c1ur',
            'hrc_bbox',
            'hrc_boundcc',
            'hrc_coord',
            'hgc_bbox',
            'hgc_boundcc',
            'hgc_coord',
            'hgc_x',
            'hgc_y',
            'hgs_bbox',
            'hgs_boundcc',
            'hgs_coord',
            'hgs_x',
            'hgs_y',
            'hpc_bbox',
            'hpc_boundcc',
            'hpc_coord',
            'hpc_geom',
            'hpc_radius',
            'hpc_x',
            'hpc_y',
            'hrc_a',
            'hrc_bbox',
            'hrc_boundcc',
            'hrc_coord',
            'hrc_r',
        ]]
    AR_data_selected = AR_data_selected.dropna(axis=0, subset=['ar_noaanum'])
    AR_data_selected = AR_data_selected.reset_index().drop(['index'], 1)
    AR_data_selected.to_csv('{0}SDO-AR-{1}.csv'.format(directory, str(date_values[d].to_period('M'))))
    print "Done AR, {0}".format(str(date_values[d].to_period('M')))

In [6]:
%%time
goes_data = pd.DataFrame.from_dict(get_goes_data(time_range))

Wall time: 8.75 s


In [7]:
goes_data['flux'] = goes_data.goes_class.apply(lambda x: goes.flareclass_to_flux(x))

In [9]:
goes_data[:3]

,end_time,event_date,goes_class,goes_location,noaa_active_region,peak_time,start_time,flux
0,2016-01-01 04:51:00,2016-01-01,B6.2,"(0, 0)",12477,2016-01-01 04:41:00,2016-01-01 04:38:00,6.2e-07 W / m2
1,2016-01-01 06:43:00,2016-01-01,C2.3,"(67, -19)",12473,2016-01-01 06:38:00,2016-01-01 06:33:00,2.3e-06 W / m2
2,2016-01-01 09:56:00,2016-01-01,C2.4,"(71, -20)",12473,2016-01-01 09:48:00,2016-01-01 09:36:00,2.4e-06 W / m2


In [24]:
(goes_data.loc[0, 'flux'])

<Quantity 6.2e-07 W / m2>

In [25]:
goes_data.noaa_active_region.value_counts()

12480    35
12488    17
0        16
12489    13
12487     8
12484     7
12473     6
12483     5
12490     4
12485     3
12476     3
12481     2
12482     1
12479     1
12478     1
12477     1
Name: noaa_active_region, dtype: int64

#Get AR data

C:\Users\Akared\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [12]:
%%time
tstart = '2016/01/01 00:00'
tend = '2016/02/01 00:00'


Wall time: 4min 5s


In [13]:

AR_data[:3]

,SOL_standard,absnetcurrenthelicity,active,ar_axislength,ar_compactnesscls,ar_lengthunit,ar_mcintoshcls,ar_mtwilsoncls,ar_neutrallength,ar_noaaclass,...,sharp_noaa_ars,sum_overlap_scores,totalenergydensityunit,totalphotoenergy,totalphotoenergydensity,totalphotoenergyunit,twistunit,unsignedcurrenthelicity,unsignedflux,unsignedvertcurrent
0,SOL2015-12-31T20:00:00L326C085,7.069000,true,None,,,,,None,,...,12472,0.968438855073890936,ergs/cm,1.643104e+30,None,ergs,1/mm,681.268005,17775.169922,1.619847e+13
1,SOL2015-12-31T20:00:00L332C111,203.585007,true,None,,,,,None,,...,12473,1.07421527314572618,ergs/cm,9.675007e+30,None,ergs,1/mm,1672.811035,33016.328125,3.264197e+13
2,SOL2015-12-31T20:00:00L241C079,1.139000,true,None,,,,,None,,...,,0,ergs/cm,3.251704e+28,None,ergs,1/mm,18.312000,254.977600,3.841124e+11


In [15]:
for i in AR_data.columns:
    print i

SOL_standard
absnetcurrenthelicity
active
ar_axislength
ar_compactnesscls
ar_lengthunit
ar_mcintoshcls
ar_mtwilsoncls
ar_neutrallength
ar_noaaclass
ar_noaanum
ar_numspots
ar_penumbracls
ar_pilcurvature
ar_polarity
ar_spotarearaw
ar_spotarearawuncert
ar_spotarearawunit
ar_spotarearepr
ar_spotarearepruncert
ar_spotareareprunit
ar_sumnegsignedflux
ar_sumpossignedflux
ar_zurichcls
area_atdiskcenter
area_atdiskcenteruncert
area_raw
area_uncert
area_unit
bound_ccnsteps
bound_ccstartc1
bound_ccstartc2
bound_chaincode
boundbox_c1ll
boundbox_c1ur
boundbox_c2ll
boundbox_c2ur
chaincodetype
comment_count
concept
currentdensityunit
currenthelicityunit
currentunit
event_avg_rating
event_c1error
event_c2error
event_clippedspatial
event_clippedtemporal
event_coord1
event_coord2
event_coord3
event_coordsys
event_coordunit
event_description
event_endtime
event_expires
event_importance
event_importance_num_ratings
event_mapurl
event_maskurl
event_npixels
event_pixelunit
event_probability
event_score
even

In [27]:
AR_data_selected = AR_data[[
        'ar_noaanum',
        'ar_mcintoshcls', 
        'search_frm_name',
        'search_instrument',
        'search_observatory',
        'sharp_noaa_ars',
        'obs_instrument',
        'obs_lastprocessingdate',
        'obs_levelnum',
        'obs_meanwavel',
        'obs_observatory',
        'obs_title',
        'obs_wavelunit',
        'event_coord1',
        'event_coord2',
        'boundbox_c1ll',
        'boundbox_c1ur',
        'hrc_bbox',
        'hrc_boundcc',
        'hrc_coord',
        'hgc_bbox',
        'hgc_boundcc',
        'hgc_coord',
        'hgc_x',
        'hgc_y',
        'hgs_bbox',
        'hgs_boundcc',
        'hgs_coord',
        'hgs_x',
        'hgs_y',
        'hpc_bbox',
        'hpc_boundcc',
        'hpc_coord',
        'hpc_geom',
        'hpc_radius',
        'hpc_x',
        'hpc_y',
        'hrc_a',
        'hrc_bbox',
        'hrc_boundcc',
        'hrc_coord',
        'hrc_r',
    ]]
AR_data_selected = AR_data_selected.dropna(axis=0, subset=['ar_noaanum'])
AR_data_selected[:3]

,ar_noaanum,ar_mcintoshcls,search_frm_name,search_instrument,search_observatory,sharp_noaa_ars,obs_instrument,obs_lastprocessingdate,obs_levelnum,obs_meanwavel,...,hpc_coord,hpc_geom,hpc_radius,hpc_x,hpc_y,hrc_a,hrc_bbox,hrc_boundcc,hrc_coord,hrc_r
0,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(730.93032 126.802698),0103000000010000000500000071C971A7F4538340BC57...,741.847731623802702,730.93032,126.802698,279.841791,"POLYGON((0.634395 271.432321,0.903503 269.8930...",,POINT(0.760681474682731 279.841790983815),0.760681
1,12473.0,,HMI SHARP,HMI,SDO,12473,HMI,,NaN,0.00005,...,POINT(741.85422 -331.472448),01030000000100000005000000D0D6C1C11EC781403FC7...,812.540255935003529,741.85422,-331.472448,245.924137,"POLYGON((0.727536 233.301616,0.932699 241.1062...",,POINT(0.833168713437757 245.924136782866),0.833169
8,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(751.9668 129.13407),0103000000010000000500000071AC8BDBE8348440BF9A...,762.974230454086182,751.96680,129.134070,279.744267,"POLYGON((0.665023 274.438474,0.916235 272.1049...",,POINT(0.782342969161661 279.744267271899),0.782343


In [30]:
AR_data_selected.reset_index().drop(['index'], 1)[:3]

,ar_noaanum,ar_mcintoshcls,search_frm_name,search_instrument,search_observatory,sharp_noaa_ars,obs_instrument,obs_lastprocessingdate,obs_levelnum,obs_meanwavel,...,hpc_coord,hpc_geom,hpc_radius,hpc_x,hpc_y,hrc_a,hrc_bbox,hrc_boundcc,hrc_coord,hrc_r
0,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(730.93032 126.802698),0103000000010000000500000071C971A7F4538340BC57...,741.847731623802702,730.93032,126.802698,279.841791,"POLYGON((0.634395 271.432321,0.903503 269.8930...",,POINT(0.760681474682731 279.841790983815),0.760681
1,12473.0,,HMI SHARP,HMI,SDO,12473,HMI,,NaN,0.00005,...,POINT(741.85422 -331.472448),01030000000100000005000000D0D6C1C11EC781403FC7...,812.540255935003529,741.85422,-331.472448,245.924137,"POLYGON((0.727536 233.301616,0.932699 241.1062...",,POINT(0.833168713437757 245.924136782866),0.833169
2,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(751.9668 129.13407),0103000000010000000500000071AC8BDBE8348440BF9A...,762.974230454086182,751.96680,129.134070,279.744267,"POLYGON((0.665023 274.438474,0.916235 272.1049...",,POINT(0.782342969161661 279.744267271899),0.782343


In [18]:
set(AR_data_selected.search_observatory)

{u'SDO', u'nil'}